In [66]:
import pandas as pd
import os
import folium

In [40]:
folder_path = "./data/flight_list/2022/"

# 抓資料夾下所有 parquet 檔案
files = [f for f in os.listdir(folder_path) if f.endswith(".parquet")]

dfs = []

for file in files:
    file_path = os.path.join(folder_path, file)
    try:
        df = pd.read_parquet(file_path)
        dfs.append(df)
    except Exception as e:
        print(f"⚠️ 無法讀取 {file}: {e}")

# 合併成一個 DataFrame（只合併成功讀取的檔案）
full_df = pd.concat(dfs, ignore_index=True)

print(f"成功合併 {len(dfs)} 個檔案")
print(full_df.shape)
print(full_df.head())

成功合併 12 個檔案
(13390934, 17)
                                                  id  icao24    flt_id  \
0  174f6dd5074bbd3d6fad9407703b4b2054f1dab8788a2d...  440a8c  TAY8151    
1  ddfcc8c8549f1843ea281b28cdedcf1b9ebfb99ffedf2a...  406d4c  VIR364     
2  6aec7e5c404cafca05ee14b9d15a114e3cc171c0bc2733...  392adb  RX19       
3  ec781dc3bca30d8c2c96ce46ac356887ad495fe3d14e86...  490031      None   
4  90019a408b4ccd08b14ec39581e3e449f79100144473ae...  77058f  ALK504     

         dof  adep  ades adep_p ades_p registration                    model  \
0 2022-01-01  None  None   None   None       OE-IFM              B747-4KZ(F)   
1 2022-01-01  None  None   None   None       G-VCRU  BOEING 787-9 Dreamliner   
2 2022-01-01  None  None   None   None         RX19                     None   
3 2022-01-01  None  None   None   None          TWR                     None   
4 2022-01-01  None  None   None   None       4R-ALO                A330 343E   

  typecode icao_aircraft_class icao_operator   

In [42]:
# 方法2：如果你也想把空字串 '' 當作空值，一起處理
df_cleaned = full_df[(full_df['adep'].notna()) & (full_df['ades'].notna()) & 
                     (full_df['adep'] != '') & (full_df['ades'] != '')]

# 看一下結果
print(df_cleaned.shape)
print(df_cleaned[['flt_id', 'adep', 'ades']].head())

(4376444, 17)
       flt_id  adep  ades
141  ZXP24     EHDL  EHGR
279  PBW13     EDNY  ETHL
284  CHX11     EDTD  LSZH
286  BOX195    EGNX  EDDP
289  THY8AH    UUWW  LTFM


In [44]:
print(f"原本資料量：{full_df.shape}")
print(f"清理後資料量：{df_cleaned.shape}")

原本資料量：(13390934, 17)
清理後資料量：(4376444, 17)


In [52]:
# 讀入機場經緯度對照表
airport_df = pd.read_csv('iata-icao.csv')  # 這裡換成你的檔名！

# 保留只需要的欄位
airport_df = airport_df[['icao', 'latitude', 'longitude']]

# 先清理 full_df：只留有 adep 和 ades 的
flights = df_cleaned.dropna(subset=['adep', 'ades'])

# 將出發地 (adep) 加上經緯度
flights = flights.merge(airport_df, how='left', left_on='adep', right_on='icao')
flights = flights.rename(columns={'latitude': 'pt1_lat', 'longitude': 'pt1_lon'})
flights = flights.drop(columns=['icao'])

# 將抵達地 (ades) 加上經緯度
flights = flights.merge(airport_df, how='left', left_on='ades', right_on='icao')
flights = flights.rename(columns={'latitude': 'pt2_lat', 'longitude': 'pt2_lon'})
flights = flights.drop(columns=['icao'])

# 只留下我們需要的欄位
flights = flights[['pt1_lat', 'pt1_lon', 'pt2_lat', 'pt2_lon']]

flights



,pt1_lat,pt1_lon,pt2_lat,pt2_lon
0,NaN,NaN,51.5674,4.93183
1,47.6713,9.51149,NaN,NaN
2,NaN,NaN,47.4647,8.54917
3,52.8311,-1.32806,51.4239,12.23640
4,55.5979,37.27090,41.2753,28.75190
...,...,...,...,...
4421966,50.8659,7.14274,50.1008,14.26000
4421967,49.0097,2.54778,52.4611,9.68508
4421968,50.9014,4.48444,NaN,NaN
4421969,NaN,NaN,49.4987,11.07810


In [56]:
# 方法2：如果你也想把空字串 '' 當作空值，一起處理
flights_cleaned = flights[(flights['pt1_lat'].notna()) & (flights['pt2_lat'].notna()) & 
                     (flights['pt1_lat'] != '') & (flights['pt2_lat'] != '')]

print(f"原本資料量：{flights.shape}")
print(f"清理後資料量：{flights_cleaned.shape}")

原本資料量：(4421971, 4)
清理後資料量：(4092479, 4)


In [58]:
# 方向不敏感：對經緯度組合排序
def sort_coords(row):
    pt1 = (row['pt1_lat'], row['pt1_lon'])
    pt2 = (row['pt2_lat'], row['pt2_lon'])
    return sorted([pt1, pt2])

flights_cleaned[['sorted_pt1', 'sorted_pt2']] = flights_cleaned.apply(lambda row: pd.Series(sort_coords(row)), axis=1)

# 整理出新 DataFrame
flight_counts = flights_cleaned.groupby(['sorted_pt1', 'sorted_pt2']).size().reset_index(name='count')

# 拆開 sorted_pt1 / sorted_pt2 回成經緯度欄位
flight_counts[['pt1_lat', 'pt1_lon']] = pd.DataFrame(flight_counts['sorted_pt1'].tolist(), index=flight_counts.index)
flight_counts[['pt2_lat', 'pt2_lon']] = pd.DataFrame(flight_counts['sorted_pt2'].tolist(), index=flight_counts.index)

# 只保留需要的欄位
final_df = flight_counts[['count', 'pt1_lat', 'pt1_lon', 'pt2_lat', 'pt2_lon']]

# 看結果
print(final_df.head())


C:\Users\PeggyTsai\AppData\Local\Temp\ipykernel_96588\352826787.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flights_cleaned[['sorted_pt1', 'sorted_pt2']] = flights_cleaned.apply(lambda row: pd.Series(sort_coords(row)), axis=1)
C:\Users\PeggyTsai\AppData\Local\Temp\ipykernel_96588\352826787.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flights_cleaned[['sorted_pt1', 'sorted_pt2']] = flights_cleaned.apply(lambda row: pd.Series(sort_coords(row)), axis=1)


   count  pt1_lat  pt1_lon  pt2_lat  pt2_lon
0      6  27.8148 -17.8871  28.4847 -16.3439
1    122  27.9319 -15.3866  28.0460 -16.5728
2      8  27.9319 -15.3866  28.4527 -13.8638
3      7  27.9319 -15.3866  28.4847 -16.3439
4      3  27.9319 -15.3866  28.6265 -17.7556


In [60]:
final_df.nlargest(10, 'count')

,count,pt1_lat,pt1_lon,pt2_lat,pt2_lon
5592,14899,39.5517,2.738810,41.2971,2.07846
34075,13445,60.1939,11.100400,60.2934,5.21814
11355,13295,43.6291,1.363820,48.7231,2.35944
5582,12796,39.5517,2.738810,40.4719,-3.56264
6184,11364,40.4719,-3.562640,41.2971,2.07846
5056,10845,38.8729,1.373120,39.5517,2.73881
4829,10812,38.7813,-9.135920,40.4719,-3.56264
11560,10524,43.6584,7.215870,48.7231,2.35944
33326,10333,55.6179,12.656000,60.1939,11.10040
29313,10112,51.4775,-0.461389,53.4213,-6.27007


In [74]:
# 取出前 N 條（次數最多的）
top_routes = final_df.sort_values(by='count', ascending=False).head(10000)
top_routes.to_csv("top_route_counts_2022.csv", index=False)

In [76]:
# 建立地圖（用第一條作為中心）
m = folium.Map(location=[top_routes['pt1_lat'].iloc[0], top_routes['pt1_lon'].iloc[0]], zoom_start=10)

for _, row in top_routes.iterrows():
    start = (row['pt1_lat'], row['pt1_lon'])
    end = (row['pt2_lat'], row['pt2_lon'])

    folium.PolyLine(
        locations=[start, end],
        color="blue",
        weight=max(1, row['count'] / top_routes['count'].max() * 8),
        opacity=0.6
    ).add_to(m)

# 存檔
m.save("top_routes_map_2022.html")